In [2]:
import sys
!pip install fpdf 
!pip install plotly-express
!pip install kaleido==0.1.0.post1


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nuzha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nuzha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/56.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/56.0 MB 660.6 kB/s eta 0:01:25
   ---------------------------------------- 0.1/56.0 MB 825.8 kB/s eta 0:01:08
   ---------------------------------------- 0.2/56.0 MB 1.5 MB/s eta 0:00:38
   ---------------------------------------- 0.4/56.0 MB 3.1 MB/s eta 0:00:19
    --------------------------------------- 1.2/56.0 MB 6.1 MB/s eta 0:00:10
   - -------------------------------------- 1.9/56.0 MB 8.1 MB/s eta 0:00:07
   - -------------------------------------- 2.7/56.0 MB 9.5 MB/s eta 0:00:06
   -- ------------------------------------- 3.3/56.0 MB 10.6 MB/s eta 0:00:05
   -- ------------------------------------- 4.1/56.0 MB 11.5 MB/s eta 0:00:05
   --- ------------------------------------ 5.0/56.0 MB 11.9 MB/s eta 0:00:05
   ---- ----------------------------------- 5.6/56.0 MB 12.4 MB/s eta 0:00:05
   ---- ----------------------------------- 6.4/56.0 MB 12.8 MB/s eta 0:00:


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: C:\Users\nuzha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## **Define paths and chart style**


In [3]:
#Installing Libraries
from datetime import date
from pathlib import Path
import sqlite3

import pandas as pd
import plotly.express as px
from fpdf import FPDF


In [4]:
plotly_template = "presentation"

In [5]:
# Define the paths
current_dir = Path(__file__).parent if "__file__" in locals() else Path.cwd()
database_path = current_dir / "nike_sales.db"
output_dir = current_dir / "output"

# Create the output directory and its parent directory if they do not exist
output_dir.mkdir(parents=True, exist_ok=True)

## **Total Sales by Month**

In [6]:

# Create a connection to the database
conn = sqlite3.connect(database_path)

In [7]:
# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT sale_date, SUM(total_price) as total_sales
FROM sales
GROUP BY sale_date
ORDER BY sale_date ASC

'''

df = pd.read_sql(query, conn)
df.head()

,sale_date,total_sales
0,2023-01-01,1590
1,2023-01-02,2360
2,2023-01-03,1690
3,2023-01-04,3220
4,2023-01-05,2000


In [8]:

# Check the data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   sale_date    341 non-null    object
 1   total_sales  341 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 5.5+ KB


In [9]:
#converts sale_date to datetime
df['sale_date'] = pd.to_datetime(df['sale_date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 341 entries, 0 to 340
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   sale_date    341 non-null    datetime64[ns]
 1   total_sales  341 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 5.5 KB


In [10]:
#set the date sale date column as the index
df.set_index('sale_date', inplace=True)
df.head()

,total_sales
sale_date,
2023-01-01,1590
2023-01-02,2360
2023-01-03,1690
2023-01-04,3220
2023-01-05,2000


In [11]:
#Resa,ple the data to a monthly frequency and sum the total sales
df_monthly = df.resample('M').sum()
df_monthly.head()

,total_sales
sale_date,
2023-01-31,58150
2023-02-28,54490
2023-03-31,75310
2023-04-30,50560
2023-05-31,59110


In [12]:
#map the month number to the month name
df_monthly['month'] = df_monthly.index.strftime('%b')
df_monthly.head()

,total_sales,month
sale_date,,
2023-01-31,58150,Jan
2023-02-28,54490,Feb
2023-03-31,75310,Mar
2023-04-30,50560,Apr
2023-05-31,59110,May


In [19]:
#create a plotly figure 
fig = px.bar(df_monthly, x='month', y='total_sales', template=plotly_template, title='Total Sales per Month')

# Define customizable font sizes
title_font_size = 24
axis_title_font_size = 18
tick_font_size = 14

fig.update_layout(
    title=dict(
        text='Total Sales per Month',
        font=dict(
            size=title_font_size  
        )
    ),
    xaxis_title=dict(
        text='Month',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    yaxis_title=dict(
        text='Total Sales',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    margin=dict(l=100),  
    yaxis=dict(
        tickprefix='$',
        tickformat=',',
        tickfont=dict(
            size=tick_font_size  
        )
    ),
    xaxis=dict(
        tickfont=dict(
            size=tick_font_size  )
    ),
    plot_bgcolor='rgba(0,0,0,0)'  
)

# Show the figure
fig.show()


# Save the figure as an image
fig.write_image(output_dir / "total_sales_per_month.png", width = 800, height = 400, scale = 2)

## **Total Sales by Product**


In [14]:
# execute the query and load the results into a pandas dataframe

query = '''
SELECT p.product_name, SUM(s.quantity) as total_sales
FROM sales s
JOIN products p ON s.product_id = p.product_id 
GROUP BY p.product_name
'''
df = pd.read_sql_query(query, conn)
df.head()

,product_name,total_sales
0,Nike Air Force 1,1028
1,Nike Air Max,1271
2,Nike Blazer Mid,1093
3,Nike React Infinity Run,1008
4,Nike ZoomX Vaporfly,957


In [20]:

# Create the Plotly figure with text parameter
fig = px.bar(df, x='product_name', y='total_sales',template=plotly_template,text='total_sales')

# Define customizable font sizes
title_font_size = 24
axis_title_font_size = 18
tick_font_size = 14

# Set the layout with customizable font sizes
fig.update_layout(
    title=dict(
        text='Total Sales by Product',
        font=dict(
            size=title_font_size  
        )
    ),
    xaxis_title=dict(
        text='Product',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    yaxis_title=dict(
        text='Total Sales ($)',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    xaxis=dict(
        tickfont=dict(
            size=tick_font_size  
        )
    ),
    yaxis=dict(
        tickfont=dict(
            size=tick_font_size  
        ),
        tickprefix='$'
    ),
    margin=dict(l=100)  
)

# Show the figure
fig.show()


# Save the chart as a PNG image
fig.write_image(output_dir / 'product_sales.png', width=1200,height=400, scale=2)

## **Top Customer by Sales**

In [23]:
#execute the query and load the results into a pandas dataframe
query = '''
SELECT c.first_name || ' ' || c.last_name as customer_name, SUM(s.total_price) as total_sales
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY customer_name
ORDER BY total_sales DESC
LIMIT 10
'''
df = pd.read_sql_query(query, conn)
df.head()

,customer_name,total_sales
0,Emma Williams,98710
1,Lucas Moore,87590
2,Noah Brown,84300
3,Mia Jackson,78720
4,Mason Lopez,74250


In [24]:
# Create the Plotly figure
fig = px.bar(df, x='customer_name',y='total_sales',template=plotly_template,text='total_sales')

title_font_size = 24
axis_title_font_size = 18
tick_font_size = 14

fig.update_layout(
    title=dict(
        text='Top Customers by Sales',
        font=dict(
            size=title_font_size  
        )
    ),
    xaxis_title=dict(
        text='Customer',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    yaxis_title=dict(
        text='Total Sales ($)',
        font=dict(
            size=axis_title_font_size  
        )
    ),
    xaxis=dict(
        tickfont=dict(
            size=tick_font_size  
        )
    ),
    yaxis=dict(
        tickfont=dict(
            size=tick_font_size 
        ),
        tickprefix='$'
    ),
    margin=dict(l=100)  
)


# Show the plot
fig.show()
# Save the chart as a PNG image
fig.write_image(output_dir / 'customer_sales.png',width=1200, height=400,scale=2)

## **Create PDF Report**


In [25]:
#define the font color
font_color = (64,64,64)

#find png files in the output directory
images = [file for file in output_dir.iterdir() if file.suffix == '.png']

#create a PDF object
pdf = FPDF()
pdf.add_page()
pdf.set_font('Arial', 'B', 16)

#add a title to the PDF
title = f"Sales Report as of {date.today().strftime('%m/%d/%Y')}"
pdf.set_text_color(*font_color)
pdf.cell(200, 10, title, ln=True, align='C')

#add each chart to the PDF
for image in images:
    pdf.image(str(image), x=10, w=190)
    pdf.ln(100)

#save the PDF
pdf.output(output_dir / 'sales_report.pdf')

''

## **Customer Segment**

In [26]:
## **executing the query and loading the results into a pandas dataframe**
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales,
  CASE 
    WHEN SUM(sales.total_price) > 30000 THEN 'High Value'
    WHEN SUM(sales.total_price) > 26000 THEN 'Medium Value'
    ELSE 'Low Value'
  END as customer_segment
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
ORDER BY total_sales DESC
'''

df = pd.read_sql_query(query, conn)
df.head()

,customer_id,customer_name,total_sales,customer_segment
0,2,Emma Williams,98710,High Value
1,8,Lucas Moore,87590,High Value
2,3,Noah Brown,84300,High Value
3,9,Mia Jackson,78720,High Value
4,10,Mason Lopez,74250,High Value


In [27]:

# Execute the query and load results into a Pandas DataFrame
query = '''
SELECT 
  customers.customer_id, 
  customers.first_name || ' ' || customers.last_name as customer_name, 
  SUM(sales.total_price) as total_sales
FROM sales
INNER JOIN customers ON sales.customer_id = customers.customer_id
GROUP BY customers.customer_id
'''

df = pd.read_sql_query(query, conn)
df.head()

,customer_id,customer_name,total_sales
0,1,Liam Johnson,67340
1,2,Emma Williams,98710
2,3,Noah Brown,84300
3,4,Olivia Taylor,55740
4,5,Ava Anderson,62230


In [28]:

# Group the data by customer segment
bins = [0, 26000, 30000, float('inf')]
labels = ['Low Value', 'Medium Value', 'High Value']
df['customer_segment'] = pd.cut(df['total_sales'], bins=bins, labels=labels)

# Order the data by total sales
df = df.sort_values(by='total_sales', ascending=False)
df.head()

,customer_id,customer_name,total_sales,customer_segment
1,2,Emma Williams,98710,High Value
7,8,Lucas Moore,87590,High Value
2,3,Noah Brown,84300,High Value
8,9,Mia Jackson,78720,High Value
9,10,Mason Lopez,74250,High Value


In [29]:

# Close the connection
conn.close()